In [ ]:
%load_ext autoreload

# (*) 以数据集形式同步读取AOP和COP，默认做对齐和ISP处理

- TianmoucDataReader 的測試功能
    - ## [输入]
    - aop_denoise, 是否開啓AOP去噪
    - aop_denoise_args = denoise_defualt_args(), 去噪默認參數
        

# 对比去噪和不去噪的结果

In [ ]:
%autoreload
import torch
import matplotlib.pyplot as plt

from tianmoucv.data import TianmoucDataReader
from tianmoucv.isp import vizDiff
from tianmoucv.data.denoise_utils import denoise_defualt_args

aim = 4 # 读取的目标COP帧的帧号
N = 1   # read continue N frames （连续读取N帧COP以及对应的AOP）

datapath = '/data/lyh/tianmoucData/20240930_tobi_sup_exp/data/lowlight_highspeed/tianmouc/100lux/1515fps_opt' #灰阶图
key_list = ['1500rpm_g4']

for key in key_list:

    # 默认关闭去噪
    dataset = TianmoucDataReader(datapath,matchkey=key,aop_denoise = False,camera_idx=0)

    # 开启去噪
    defualt_args = denoise_defualt_args()
    denoised_dataset = TianmoucDataReader(datapath,matchkey=key,aop_denoise = True, aop_denoise_args = defualt_args, camera_idx=0)
    
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        print(sample['meta'])
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0

            tsd_rgb = tsdiff[:,i,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)
            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()

    print('去噪结果')
    for index in range(aim,min(aim+1,len(denoised_dataset))):
        sample = denoised_dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0
            tsd_rgb = tsdiff[:,i,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()
        

# 如果录制了暗电流噪声，可以使用完整版本

In [ ]:
%autoreload

from tianmoucv.isp import vizDiff
from tianmoucv.data.denoise_utils import denoise_defualt_args

# 用于生成需要的Dark Noise Array
dark_noise_path = '/data/lyh/tianmoucData/20240930_tobi_sup_exp/data/tianmouc_dark/dark_gain4'

def aop_denoise_arg_generator(dark_noise_path = None):
    aop_gain = 4 # 注意这个
    aop_denoise_args= denoise_defualt_args()
    aop_denoise_args.thr_1=2 * aop_gain
    aop_denoise_args.thr_2=5 * aop_gain
    aop_denoise_args.thr_3=5 * aop_gain
    aop_denoise_args.gain = aop_gain        
    dataset_dark = TianmoucDataReader(dark_noise_path,aop_denoise = True)
    TD_dark, SDL_dark, SDR_dark = dataset_dark.choose_correct_fpn()
    aop_denoise_args.aop_dark_dict = {'TD':TD_dark,'SDL':SDL_dark,'SDR':SDR_dark}
    print(len(SDR_dark))
    return aop_denoise_args,aop_gain

N = 1
for key in key_list:
    # 开启去噪
    aop_denoise_args,aop_gain = aop_denoise_arg_generator(dark_noise_path)
    denoised_dataset = TianmoucDataReader(datapath,matchkey=key,
                                          aop_denoise = True, 
                                          aop_denoise_args = aop_denoise_args, 
                                          camera_idx=0)

    print('去噪结果')
    for index in range(aim,min(aim+5,len(denoised_dataset))):
        sample = denoised_dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0

            tsd_rgb = tsdiff[:,12,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)
            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()
        